In [82]:
import pandas as pd
import re
from sqlalchemy import create_engine
from datetime import datetime , timedelta
from dateutil.relativedelta import relativedelta

In [2]:
path = 'D:/AS/'
state = 'AS'

In [4]:
vt_owner = pd.read_csv(path+'/vt_owner.csv')
vt_owner = vt_owner[vt_owner['State Cd']==state]

In [142]:
vt_fee = pd.read_csv(path+'/vt_fee.csv')
vt_fee = vt_fee[vt_fee['Pur Cd']==1]
vt_fee_1 = vt_fee[vt_fee['Off Cd']==1]
vt_fee_1.reset_index(drop=True, inplace=True)

In [145]:
vt_fee_1['Rcpt Dt2'] = vt_fee_1.loc[:,'Rcpt Dt'].apply(lambda x: datetime.strptime(x[:11], '%d-%b-%Y'))

C:\Users\CDMA\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [149]:
df1 = vt_fee_1[vt_fee_1['Rcpt Dt2'] > dt]
df2 = vt_fee_1[vt_fee_1['Rcpt Dt2'] < dt]
#vt_fee_1 = pd.concat([df1,df2])
vt_fee_1 = df1

In [110]:
vm_feemast = pd.read_csv(r'C:\Users\CDMA\Downloads\vm_feemast.csv')
vm_feemast=vm_feemast[vm_feemast['pur_cd']==1]
vm_feemast=vm_feemast[vm_feemast['state_cd']==state]
vm_feemast.head()

,pur_cd,fees,vh_class,service_charge,state_cd
0,7,0,0,0,PY
1,2,300,57,0,DL
2,15,300,57,0,DL
3,6,500,57,0,DL
4,3,150,57,0,DL


In [29]:
vm_feemast_catg = pd.read_csv(r'C:\Users\CDMA\Downloads\vm_feemast_catg.csv')
vm_feemast_catg=vm_feemast_catg[vm_feemast_catg['pur_cd']==1]
vm_feemast_catg=vm_feemast_catg[vm_feemast_catg['state_cd']=='AS']
vm_feemast_catg['vch_catg']=vm_feemast_catg['vch_catg'].str.strip()
vm_feemast_catg.head()

In [116]:
vm_feemast_penalty = pd.read_csv(r'C:\Users\CDMA\Downloads\vm_feemast_penalty.csv')
vm_feemast_penalty['from_dt'] = vm_feemast_penalty['from_dt'].astype('datetime64[ns]')
vm_feemast_penalty['upto_dt'] = vm_feemast_penalty['upto_dt'].astype('datetime64[ns]')
vm_feemast_penalty.reset_index(drop=True, inplace=True)
vm_feemast_penalty.head()

,Number of Records,condition_formula,day_from,day_upto,fine_amt,from_dt,grace_days,grace_unit_days,is_percent_rate,max_amount,penalty_mode,per_unit,pur_cd,sr_no,state_cd,upto_dt
0,1,True,0,7,35,1900-01-01,7,0,False,0,F,0,1,1,AS,2100-01-01
1,1,True,1,9999,5,1900-01-01,7,0,False,0,D,1,1,2,AS,2100-01-01
2,1,True,1,9999,50,2016-12-29,0,0,False,0,D,1,2,1,AS,2100-01-01


In [58]:
conn_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(user='postgres',
                                                                     pwd='123',
                                                                     host='localhost',
                                                                     port='5432',
                                                                     db='postgres'
                                                                     )
engine = create_engine(conn_string, paramstyle='format', isolation_level="READ UNCOMMITTED")

In [164]:
actual_fee = []
for i in range(0,50):     
    regn_no = vt_fee_1['Regn No'].to_list()[i] 
    
    state_cd = vt_fee_1['State Cd'].to_list()[i]
    fee_paid = vt_fee_1['Fees'].to_list()[i]
    fine_paid = vt_fee_1['Fine'].to_list()[i]
    rcpt_dt = datetime.strptime(vt_fee_1['Rcpt Dt'].to_list()[i][:11],'%d-%b-%Y')
    
    vt_owner_nw = vt_owner[vt_owner['Regn No']==regn_no]
    #print(vt_owner_nw)
    vh_class = vt_owner_nw['Vh Class'].to_list()
    if len(vh_class)>0:
        vh_class = vh_class[0]
    else:
        print('No Info in vt_owner',regn_no,rcpt_dt)
        continue
    vch_catg = vt_owner_nw['Vch Catg'].to_list()[0]
    feemast_catg = vm_feemast_catg[vm_feemast_catg['vch_catg']==vch_catg]
    cf_params = []
    for k in re.split(r'[()\*\s]\s*',cf):
        if '<' in k and '>' in k:
            cf_params.append(k[k.find('<'):(k.find('>')+1)])
    cf_params = list(set(cf_params))
    c_formula = feemast_catg['condition_formula'].to_list()[0]
    for j in cf_params:
        if j==94:
            p_value = vt_fee_1[vt_fee_1['Regn No']==regn_no]['Off Cd'].to_list()[0]
        elif j==38:
            p_value = vch_catg
        elif j ==3:
            p_value = vh_class
        elif j ==28:
            p_value = vt_permit['pmt_catg'].to_list()[0]
        elif j ==20:
            p_value = vt_owner_nw['Owner Cd'].to_list()[0]
        c_formula = c_formula.replace(str(m),str(p_value))
    cf_result = pd.read_sql_query(repr('select '+c_formula)[1:-1],con=engine)
    #print(cf_result)
    cf_result = cf_result.loc[0,'bool']
    if cf_result==True:
        fee_act = feemast_catg['fees'].to_list()[0]
    if fee_paid==fee_act:
        print('Matched',regn_no,str(regn_dt)[:10],str(purchase_dt)[:10],vh_class,vch_catg,cf,cf_params,fee_paid,fee_act,rcpt_dt)
    else:
        print('Mismatch',regn_no,regn_dt,purchase_dt,vh_class,vch_catg,cf,cf_params,fee_paid,fee_act,rcpt_dt)
    
    purchase_dt = datetime.strptime(vt_owner_nw['Purchase Dt'].to_list()[0][:11],'%d-%b-%Y')
    regn_dt = vt_owner_nw['Regn Dt'].to_list()[0]
    delay =  relativedelta(regn_dt, purchase_dt)
    delay = delay.years * 365 + delay.months * 30 + delay.days
    if delay>0:
        feemast_penalty = vm_feemast_penalty[(vm_feemast_penalty['state_cd']==state_cd) & (vm_feemast_penalty['from_dt'] < rcpt_dt) & (vm_feemast_penalty['upto_dt'] > rcpt_dt) & (vm_feemast_penalty['pur_cd']==1) & (vm_feemast_penalty['day_upto'] > delay) ]
        if len(feemast_penalty)>1:
            ds = min(feemast_penalty['day_upto'].to_list())
            feemast_penalty = feemast_penalty[feemast_penalty['day_upto']==ds]
        if feemast_penalty['penalty_mode'].to_list()[0]=='F':
            rate = feemast_penalty['fine_amt'].to_list()[0]
        else:
            rate = feemast_penalty['fine_amt'].to_list()[0]* delay

        print('Days Delayed',delay,'Fine',rate)





No Info in vt_owner AS01EF1565 2019-09-28 00:00:00
No Info in vt_owner NEW 2019-10-01 00:00:00
No Info in vt_owner AS01LC6515 2019-09-20 00:00:00
Matched AS01EE4198 2019-09-10 2019-08-20 1 2WN true [] 50 50 2019-09-03 00:00:00
No Info in vt_owner AS01EF2073 2019-09-30 00:00:00
No Info in vt_owner NEW 2019-09-28 00:00:00
No Info in vt_owner NEW 2019-09-28 00:00:00
Matched AS01EE4270 2019-09-03 2019-09-03 1 2WN true [] 50 50 2019-09-03 00:00:00
No Info in vt_owner AS01EF2037 2019-09-30 00:00:00
Matched AS01EE4236 2019-09-03 2019-09-03 1 2WN true [] 50 50 2019-09-03 00:00:00
No Info in vt_owner AS01EF1498 2019-09-28 00:00:00
No Info in vt_owner AS01EF1509 2019-09-28 00:00:00
No Info in vt_owner AS01EF1513 2019-09-28 00:00:00
Matched AS01EE6005 2019-09-03 2019-09-03 7 LMV true [] 600 600 2019-09-10 00:00:00
Matched AS01EE4165 2019-09-10 2019-09-10 1 2WN true [] 50 50 2019-09-03 00:00:00
No Info in vt_owner AS01EF1595 2019-09-28 00:00:00
No Info in vt_owner AS01EE9048 2019-09-20 00:00:00
No

In [120]:
vt_owner[vt_owner['Regn No']=='AS01EF1565']['Purchase Dt']

Series([], Name: Purchase Dt, dtype: object)

In [108]:
vt_owner['Regn Dt'] = vt_owner.loc[:,'Regn Dt'].apply(lambda x: datetime.strptime(x[:11], '%d-%b-%Y'))

In [105]:
vt_owner[vt_owner['Regn No']=='AS01EE6081']['Vch Catg']

2218843    LMV
Name: Vch Catg, dtype: object

In [79]:
vt_fee_1[vt_fee_1['Regn No']=='AS01EE6081'] 

,Number of Records,Collected By,Fees,Fine,Flag,Off Cd,Payment Mode,Pur Cd,Rcpt Dt,Rcpt No,Regn No,State Cd
8396189,1,1702029843,600,0,NaN,1,I,1,11-Sep-2019 13:17:49,AS1D190900002500,AS01EE6081,AS


In [78]:
vt_owner[vt_owner['Regn No']=='AS01EE3643']['Regn Dt']

2215712    02-Sep-2019
Name: Regn Dt, dtype: object

In [87]:
dt = datetime.strptime('01-Sep-2019', '%d-%b-%Y')
#vt_fee_1.dtypes

In [102]:
df = pd.read_csv

In [103]:
vt_fee_2 = vt_fee_1
vt_fee_1 = fee_df

In [134]:
vt_fee_1.shape

(10, 13)

In [166]:
df= pd.read_csv(r'C:\Users\CDMA\Downloads\DOB DISTINCT VALUES.csv')

In [168]:
df.columns

Index(['01/06/1987'], dtype='object')

In [ ]:
import re

In [ ]:
for i in df['01/06/1987']:
re.findall(i,''